In [1]:
# import dependecies 
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import datetime
import numpy as np

#Config should contain database username as username and database password as password
import config

In [2]:
# connect to SQL database 

protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)
conn = engine.connect()

In [3]:
#Fetch station data
stationData = pd.read_csv("./Resources/stationData.csv")
stationData.head()

,WMO_NUM,station_number,station_name,LATITUDE,LONGITUDE,STN_HT,AVIATION_ID,REGION,GridPt Lat,GridPt Lon,MSAS elevation,Distance from GridPt,Roughness,Distance from coast,Category,forecast_district,sa_special
0,94648,23000,ADELAIDE (WEST TERRACE / NGAYIRDAPIRA),-34.9257,138.5832,29.32,ADWT,SA,NaN,NaN,NaN,NaN,NaN,NaN,coast,SA_PW001,sag
1,94489,38076,WINDORAH AP,-25.4117,142.6647,132.16,YWDH,QLD,NaN,NaN,NaN,NaN,NaN,NaN,flat_inland,QLD_PW011,NaN
2,94795,9281,MILLENDON (SWAN VALLEY),-31.8108,116.0225,16.00,SWVA,WA,NaN,NaN,NaN,NaN,NaN,NaN,mountains2,WA_PW009,NaN
3,99218,32194,COWLEY BEACH,-17.6904,146.1126,17.00,CBTA,QLD,NaN,NaN,NaN,NaN,NaN,NaN,coast,QLD_PW004,NaN
4,94794,51164,GIRILAMBONE (OKEH),-31.0822,146.9294,178.00,NDR2,NSW,NaN,NaN,NaN,NaN,NaN,NaN,flat_inland,NSW_PW012,NaN


In [4]:
#Make sure to match schema
stationData_sql = pd.DataFrame()
stationData_sql["station_number"] = stationData["station_number"]
stationData_sql["station_name"] = stationData["station_name"]
stationData_sql["lat"] = stationData["LATITUDE"]
stationData_sql["lon"] = stationData["LONGITUDE"]
stationData_sql["height"] = stationData["STN_HT"]
stationData_sql["region"] = stationData["REGION"]
stationData_sql.to_sql("station_data", conn,if_exists='append',index=False)



In [5]:
#Get file list for fcst and obs tables
fcst_files = os.listdir("./Resources/fcst")
fcst_data = pd.DataFrame()
for file in fcst_files:
    date = datetime.datetime.strptime(file[-12:-4],"%Y%m%d")
    tempdf = pd.read_csv("./Resources/fcst/" + file)
    tempdf = tempdf.loc[(tempdf["parameter"] == "MaxT") | (tempdf["parameter"] == "MinT")].reset_index()
    first_seconds = tempdf["valid_start"][0]
    tempdf_sql = pd.DataFrame()
    tempdf_sql["station_number"] = tempdf["station_number"]
    tempdf_sql["parameter"] = tempdf["parameter"]
    tempdf_sql["area_code"] = tempdf["area_code"]
    tempdf_sql["valid_start"]  = (tempdf["valid_start"] - first_seconds)/(60*60)
    tempdf_sql["valid_end"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["temperature"]  = (tempdf["value"])
    tempdf_sql["date"] = date
    tempdf_sql.to_sql("fcst",conn,if_exists='append',index=False)

In [6]:
#Get file list for fcst and obs tables
obs_files = os.listdir("./Resources/obs")
fcst_data = pd.DataFrame()
for file in obs_files:
    date = datetime.datetime.strptime(file[-12:-4],"%Y%m%d")
    tempdf = pd.read_csv("./Resources/obs/" + file)
    tempdf = tempdf.loc[tempdf["parameter"] == "AIR_TEMP"]
    first_seconds = tempdf["valid_start"][0]
    tempdf_sql = pd.DataFrame()
    tempdf_sql["station_number"] = tempdf["station_number"]
    tempdf_sql["area_code"] = tempdf["area_code"]
    tempdf_sql["valid_start"]  = (tempdf["valid_start"] - first_seconds)/(60*60)
    tempdf_sql["valid_end"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["temperature"]  = (tempdf["value"])
    tempdf_sql["date"] = date
    tempdf_sql.to_sql("obs",conn,if_exists='append',index=False)

In [7]:
stations = pd.read_sql("SELECT * FROM station_data", con = conn)
fcst = pd.read_sql("SELECT * FROM fcst", con = conn)
obs = pd.read_sql("SELECT * FROM obs", con = conn)
full_dataset = fcst.merge(obs, how = "outer", on = ["date", "station_number", "valid_start"])

In [8]:
#Get max/min temps
daily_obs = obs.groupby(["date", "station_number"])
daily_obs.head()
extremes = pd.DataFrame()
extremes["max"] = daily_obs["temperature"].max()
extremes["min"] = daily_obs["temperature"].min()
extremes = extremes.reset_index()
extremes.head() 

,date,station_number,max,min
0,2016-05-01 00:00:00,1006,38.1,24.8
1,2016-05-01 00:00:00,1007,33.5,28.2
2,2016-05-01 00:00:00,1019,38.0,20.4
3,2016-05-01 00:00:00,1020,36.2,24.7
4,2016-05-01 00:00:00,2012,36.6,21.8


In [9]:
max_fcst = fcst.loc[fcst["parameter"] == "MaxT"]
min_fcst = fcst.loc[fcst["parameter"] == "MinT"]
max_fcst = max_fcst.groupby(["date", "station_number"])["temperature"].mean()
min_fcst = min_fcst.groupby(["date", "station_number"])["temperature"].mean()
combined_fcst = pd.DataFrame()
combined_fcst["max"] = max_fcst
combined_fcst["min"] = min_fcst
combined_fcst.reset_index()

,date,station_number,max,min
0,2016-05-01 00:00:00,1006,35.955556,23.511111
1,2016-05-01 00:00:00,1019,34.533333,20.855556
2,2016-05-01 00:00:00,1020,34.400000,20.300000
3,2016-05-01 00:00:00,2012,32.722222,20.544444
4,2016-05-01 00:00:00,2056,34.500000,21.433333
...,...,...,...,...
5486,2017-04-01 00:00:00,97085,11.760000,4.488889
5487,2017-04-01 00:00:00,98017,20.910000,13.722222
5488,2017-04-01 00:00:00,99005,21.300000,13.011111
5489,2017-04-01 00:00:00,200786,31.122222,27.388889


In [10]:
combined_fcst.head()

max        min
date                station_number                      
2016-05-01 00:00:00 1006            35.955556  23.511111
                    1019            34.533333  20.855556
                    1020            34.400000  20.300000
                    2012            32.722222  20.544444
                    2056            34.500000  21.433333

In [11]:
full = extremes.merge(combined_fcst, on=['date','station_number'])
full.head()

,date,station_number,max_x,min_x,max_y,min_y
0,2016-05-01 00:00:00,1006,38.1,24.8,35.955556,23.511111
1,2016-05-01 00:00:00,1019,38.0,20.4,34.533333,20.855556
2,2016-05-01 00:00:00,1020,36.2,24.7,34.400000,20.300000
3,2016-05-01 00:00:00,2012,36.6,21.8,32.722222,20.544444
4,2016-05-01 00:00:00,2056,37.2,22.6,34.500000,21.433333


In [12]:
variation = pd.DataFrame()
variation["station_number"] = full["station_number"]
variation["station_difference"] = (np.sqrt((full["max_x"] - full["max_y"])**2) + np.sqrt((full["min_x"] - full["min_y"])**2))/2
variation = variation.groupby("station_number")["station_difference"].mean().reset_index()

variation = variation.merge(stationData_sql, how = "left", on = "station_number")

variation.to_sql("variation", conn,if_exists='append',index=False)
variation.head()

,station_number,station_difference,station_name,lat,lon,height,region
0,1006,1.590404,WYNDHAM AERO,-15.5100,128.1503,3.8,WA
1,1019,1.152020,KALUMBURU,-14.2964,126.6453,23.0,WA
2,1020,1.516667,TRUSCOTT,-14.0900,126.3867,51.0,WA
3,2012,1.854040,HALLS CREEK AIRPORT,-18.2292,127.6636,422.0,WA
4,2056,1.537374,KUNUNURRA AERO,-15.7814,128.7100,44.0,WA


In [13]:
conn.close()